In [7]:
import numpy as np
from hwcomponents import DataStructures as ds 
from hwcomponents import Utils

In [8]:
# Test Neuron datastructure
ds.Neuron(
    param_leak_str=1,
    param_threshold=20,
    param_reset=0,
    state_core=0
)

Neuron For Memory(leak_str=1, threshold=20, reset=0, core=0)

In [9]:
# Test Synapse datastructure
ds.Synapse(
    length=10,
    word_size=np.uint8
)

Synapse For Memory(length=10, weights=[0 0 0 0 0 0 0 0 0 0])

In [10]:
# ex of random neuron generator
def random_neuron() -> ds.Neuron:
    return ds.Neuron(
        param_leak_str=Utils.DiscreteNORM(2**3, 2, 0, 2**5),
        param_threshold=Utils.DiscreteNORM(2**6, 2**3, 0, 2**11),
        state_core=0,
        param_reset=Utils.DiscreteNORM(4., 1., 0, 2**3)
    )

random_neuron()

Neuron For Memory(leak_str=5.0, threshold=61.0, reset=4.0, core=0)

In [18]:
# ex of BRAM with random neurons
from hwcomponents import BRAM as bram

amount_of_neurons = 10
neuron_generator = lambda: random_neuron()

example_bram_for_neurons = bram.Neurons(
    length=amount_of_neurons,
    randomize_fun=neuron_generator
)

for i in range(example_bram_for_neurons.length):
    print(example_bram_for_neurons[i])

Neuron For Memory(leak_str=6.0, threshold=64.0, reset=3.0, core=0)
Neuron For Memory(leak_str=11.0, threshold=69.0, reset=6.0, core=0)
Neuron For Memory(leak_str=7.0, threshold=60.0, reset=4.0, core=0)
Neuron For Memory(leak_str=11.0, threshold=65.0, reset=4.0, core=0)
Neuron For Memory(leak_str=6.0, threshold=65.0, reset=6.0, core=0)
Neuron For Memory(leak_str=8.0, threshold=78.0, reset=4.0, core=0)
Neuron For Memory(leak_str=10.0, threshold=59.0, reset=3.0, core=0)
Neuron For Memory(leak_str=7.0, threshold=70.0, reset=5.0, core=0)
Neuron For Memory(leak_str=7.0, threshold=52.0, reset=4.0, core=0)
Neuron For Memory(leak_str=10.0, threshold=62.0, reset=3.0, core=0)


In [ ]:
# ex of random synapse generator
# generate a synapse register for one neuron. 
# in this case it generates a 2-bit synaptic weight (0..3 in decimal)

def random_synapse(depth: int) -> ds.Synapse:
    temp_synapse = ds.Synapse(depth, np.int8)
    for i in range(depth):
        temp_synapse[i] = Utils.DiscreteNORM(0.5, 1., 0, 3)
    return temp_synapse

In [ ]:
# ex of BRAM with random synapses
from hwcomponents import BRAM as bram
amount_of_synapses = 10


